教程地址：https://stackoverflow.com/questions/43331510/how-to-train-an-svm-classifier-on-a-satellite-image-using-python

In [ ]:
import numpy as np
from skimage import io

img = io.imread('D:\Projects\VsCode\Python\img_processing_system\img\TFOv7.png')
print(img.shape)
rows, cols, bands = img.shape
print(rows, cols, bands)
classes = {'building': 0, 'vegetation': 1, 'water': 2}
n_classes = len(classes)
palette = np.uint8([
    [0, 255, 0], 
    [255, 0, 0], 
    [0, 0, 255]
    ])

In [ ]:
print(palette)

# 非监督式学习

In [ ]:
from sklearn.cluster import KMeans

X = img.reshape(rows*cols, bands)
print("X.shape: ",X.shape)
kmeans = KMeans(n_clusters=n_classes, random_state=3).fit(X)
print(kmeans.labels_)
print(np.unique(kmeans.labels_))
unsupervised = kmeans.labels_.reshape(rows, cols)
print(unsupervised)

In [ ]:
print(type(palette[unsupervised]))
print(palette[unsupervised].shape)
print(palette[unsupervised])

## 问题：palette与unsupervised中的分类如何对应的？

In [ ]:
io.imshow(palette[unsupervised]) 


# 监督学习

In [ ]:
supervised = n_classes * np.ones(shape=(rows, cols), dtype=int)
supervised[200:220, 150:170] = classes['building']
supervised[40:60, 40:60] = classes['vegetation']
supervised[100:120, 200:220] = classes['water']

In [ ]:
y = supervised.ravel() # 拉伸为1维
train = np.flatnonzero(supervised < n_classes) # 即找到标记为building，vegetation，water的下标
''' 你用来分析的
print(supervised < n_classes)
res = (supervised < n_classes).ravel()
idx_list = []
for i in range(0, len(res)):
    if(res[i]):
        idx_list.append(i)
print(idx_list)
print(train)
'''
test = np.flatnonzero(supervised == n_classes) # 即没有被标记为building，vegetation，water的下标


#from sklearn.svm import SVC
#clf = SVC(gamma='auto')
#clf.fit(X[train], y[train])

from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100, class_weight="balanced")
clf.fit(X[train], y[train])

y[test] = clf.predict(X[test])
supervised = y.reshape(rows, cols)

palette = np.uint8([
    [255, 0, 0], 
    [0, 255, 0],
    [0, 0, 255]
    ])

io.imshow(palette[supervised])